# **Dog-Breed Identification**

 *The dataset is downloaded from the following: https://www.kaggle.com/c/dog-breed-identification/data*
 
 **Why did I choose this particular challenge?**
 
 The reason that I chose this dataset is because it has 120 different classes of breeds among dogs. That is, it is 120 class classification problem and I have never delt with a classification model of these many classes before. So I took this opportunity to take on a challenge and see how I could push myself to go about it. I realized that the datasets include a huge number of images of different breed of dogs (120) and that it would require a decent amount of computation power to train on all the images which I couldn't possibly do in my laptop, so I used Google Colab to train the model. 


 **Challenges Faced and Approach**
 
-  The first challenge, was to figure out how to train the model on such a huge dataset, for which I used Google Colab.
 
-  The second challenge that I came across after I trained my own model, was validation accuracy. to resolve this problem, I used the transfer learning approach where I used a pre-trained model on top of my own model to achieve better accuracy.

- Consequently, I opted to use Jupyter Notebooks so that I could document everything, each step of the project, along with the code to clearly illustrate how I went about the project with the hope that it makes it easy to follow my approach to this problem and evaluate my performance.

## Step 1 - Importing all the important libraries

In [ ]:
import numpy as np 
import pandas as pd
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
import tensorflow as tf
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2

- #### The following code is to link Gdrive to Google Colab and to unzip the files.

In [ ]:
# To Authorize your Gdrive to Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# To unzip from the drive
#!unzip -q "drive/My Drive/dataset/test.zip"

In [ ]:
# To unzip from the drive
#!unzip -q "drive/My Drive/dataset/train.zip"

## Step 2 - Data Preparation

•	In this step the data is loaded from the given CSV files. <br>
•	Target variables are OneHotEncoded so as to add the final predictions into the submissions file. <br>
•	Setting up of a specific image size (90x90). <br>
•	Creating lists for train, valid and test and then reading in the image files from test and train folders provided by the dataset and appending them to the list created. <br>
•	Converting the lists containing the image data as arrays. <br>
•	Splitting up the training set into train and validation sets. <br>


- #### Reading the csv files

In [ ]:
df_train = pd.read_csv('labels.csv')
df_test = pd.read_csv('sample_submission.csv')

#### Displaying the contents of training set

In [8]:
df_train.head(5)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


 - #### Displaying the contents of test set

In [ ]:
df_test.head(2)

- #### Using OneHotEncoding so as to put the outputs in submissions file

In [ ]:
#the breed needs to be one-hot encoded for the final submission
targets = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets, sparse = True)

In [ ]:
# Converting the encoded file as array
one_hot_labels = np.asarray(one_hot)

In [11]:
one_hot_labels[99]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

- #### Reading the images from test and train folders

In [ ]:
image_size = 90

In [ ]:
train = []
valid = []
test = []

In [14]:
i = 0
#   id  breed       train set
for f, breed in tqdm(df_train.values):
    img = cv2.imread('train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    train.append(cv2.resize(img, (image_size, image_size)))
    valid.append(label)
    i += 1
    

100%|██████████| 10222/10222 [00:22<00:00, 460.57it/s]


In [15]:
for f in tqdm(df_test['id'].values):
    img = cv2.imread('test/{}.jpg'.format(f))
    test.append(cv2.resize(img, (image_size, image_size)))

100%|██████████| 10357/10357 [00:21<00:00, 473.97it/s]


- #### Converting Train, Test, Valid to Array.

In [ ]:
train_ar = np.array(train, np.float32) / 255 #Bring it to the pixel value
valid_ar = np.array(valid, np.uint8)
test_ar = np.array(test, np.float32) / 255

In [17]:
# Checking the shaleps of array elements
print(train_ar.shape)
print(valid_ar.shape)
print(test_ar.shape)

(10222, 90, 90, 3)
(10222, 120)
(10357, 90, 90, 3)


- #### As Validation set contains 120 different class items, it can separated and be used while building a CNN model.

In [ ]:
no_classes = valid_ar.shape[1]

- #### Splitting the Train and Valid sets into splits

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(train_ar, valid_ar, test_size = 0.3, random_state = 1)

In [20]:
# Verifying the shapes of the splits
print(X_train.shape)
print(X_valid.shape)
print(Y_train.shape)
print(Y_valid.shape)

(7155, 90, 90, 3)
(3067, 90, 90, 3)
(7155, 120)
(3067, 120)


## Step 3 - Building the CNN Model

•	Importing the important libraries. <br>
•	Building the Sequential model. <br>
•	Adding a convolutional layer with 64 filters followed by a max pool layer. <br>
•	Adding another conv layer with 32 filters followed by a max pool layer. <br>
•	Adding the last conv layer with 128 filters followed by a max pool layer. <br>
•	Adding a flatten layer to flatten the images. <br>
•	Then finally adding a fully connected layer. <br>
•	Epochs used are 32 inorder to watch the performance of the model. <br>


In [ ]:
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

- ### This model is trained without any pre-trained weights. The use of a very small learning rate is to reduced the validation loss through each epoch. 

    

In [ ]:
optimiser = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)


# Initialising the CNN
classifier = Sequential()

# Convolution and adding a maxpool layer
classifier.add(Conv2D(64, 3, 3, input_shape = (image_size, image_size, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
classifier.add(MaxPooling2D(pool_size=(2, 2)))


# Flattening
classifier.add(Flatten())

# Full connection 
classifier.add(Dense(units = 128, activation = 'relu'))  # "Hidden Layer"
classifier.add(Dropout(0.25))
classifier.add(Dropout(0.25))
classifier.add(Dropout(0.25))
classifier.add(Dense(units = no_classes, activation = 'softmax')) # Softmax because the outcome has a 120 classes. "OutPut Layer".

# Compiling the CNN
classifier.compile(loss='categorical_crossentropy',
                   optimizer = optimiser,
                   metrics=['accuracy'])
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]

classifier.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 88, 88, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 44, 44, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 128)       36992     
_________________________________________________________________
max_pooling2d_3 (MaxP

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(90, 90, 3..., activation="relu")`
  


- ### In order to prevent overfitting, early stopping is used to customize the learning rate on each iteration.

In [ ]:
from keras.callbacks import Callback

class TerminateOnBaseline(Callback):
    """Callback that terminates training when either acc or val_acc reaches a specified baseline
    """
    def __init__(self, monitor='acc', baseline=0.9):
        super(TerminateOnBaseline, self).__init__()
        self.monitor = monitor
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        acc = logs.get(self.monitor)
        if acc is not None:
            if acc >= self.baseline:
                print('Epoch %d: Reached baseline, terminating training' % (epoch))
                self.model.stop_training = True
                
classifier.fit(X_train, Y_train, epochs = 32, validation_data = (X_valid, Y_valid), verbose=1, callbacks=[TerminateOnBaseline(monitor='acc', baseline=0.90)])

Instructions for updating:
Use tf.cast instead.
Train on 7155 samples, validate on 3067 samples
Epoch 1/32
7155/7155 [==============================] - 7s 1ms/step - loss: 4.7899 - acc: 0.0078 - val_loss: 4.7863 - val_acc: 0.0085
Epoch 2/32
7155/7155 [==============================] - 4s 547us/step - loss: 4.7851 - acc: 0.0105 - val_loss: 4.7835 - val_acc: 0.0117
Epoch 3/32
7155/7155 [==============================] - 4s 553us/step - loss: 4.7787 - acc: 0.0124 - val_loss: 4.7691 - val_acc: 0.0170
Epoch 4/32
7155/7155 [==============================] - 4s 550us/step - loss: 4.7597 - acc: 0.0133 - val_loss: 4.7393 - val_acc: 0.0222
Epoch 5/32
7155/7155 [==============================] - 4s 552us/step - loss: 4.7274 - acc: 0.0186 - val_loss: 4.7067 - val_acc: 0.0248
Epoch 6/32
7155/7155 [==============================] - 4s 555us/step - loss: 4.6861 - acc: 0.0249 - val_loss: 4.6527 - val_acc: 0.0303
Epoch 7/32
7155/7155 [==============================] - 4s 556us/step - loss: 4.6321 - acc

## ResNet50 model with pre-trained weights.

In [ ]:
#  Unzipping the h5 weights file.
!unzip -q "drive/My Drive/dataset/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5.zip"

In [22]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Activation, Flatten, Dropout, BatchNormalization
import h5py



#resnet_weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights='resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'))
my_new_model.add(Dense(512))
my_new_model.add(Activation('relu'))
my_new_model.add(Dropout(0.5))
my_new_model.add(Dense(no_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


my_new_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
activation_49 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               61560     
Total params: 24,698,360
Trainable params: 1,110,648
Non-trainable params: 23,587,712
_________________________________________________________________


In [26]:
from keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping

stop = EarlyStopping(monitor='val_loss', patience=15, verbose=1, mode='min')


                
my_new_model.fit(X_train, Y_train, epochs = 32, validation_data = (X_valid, Y_valid), verbose=1, callbacks=[stop])

Train on 7155 samples, validate on 3067 samples
Epoch 1/32
7155/7155 [==============================] - 16s 2ms/sample - loss: 5.5056 - accuracy: 0.0080 - val_loss: 8.6706 - val_accuracy: 0.0095
Epoch 2/32
7155/7155 [==============================] - 9s 1ms/sample - loss: 5.2143 - accuracy: 0.0119 - val_loss: 9.6978 - val_accuracy: 0.0114
Epoch 3/32
7155/7155 [==============================] - 9s 1ms/sample - loss: 5.0735 - accuracy: 0.0150 - val_loss: 5.3506 - val_accuracy: 0.0078
Epoch 4/32
7155/7155 [==============================] - 9s 1ms/sample - loss: 4.9534 - accuracy: 0.0168 - val_loss: 4.9547 - val_accuracy: 0.0108
Epoch 5/32
7155/7155 [==============================] - 9s 1ms/sample - loss: 4.8653 - accuracy: 0.0203 - val_loss: 4.7809 - val_accuracy: 0.0218
Epoch 6/32
7155/7155 [==============================] - 9s 1ms/sample - loss: 4.8032 - accuracy: 0.0233 - val_loss: 4.7136 - val_accuracy: 0.0271
Epoch 7/32
7155/7155 [==============================] - 9s 1ms/sample - los

### Step 4 - Predictions

•	Predicting the values of the classifier. <br>
•	Creating a new data frame and adding predicted values. <br>
•	Setting the column names as generated from OneHotEncoding. <br>
•	Inserting the test set column id to the final submission file. <br>



In [28]:
preds = my_new_model.predict(test_ar, verbose=1)

10357/10357 [==============================] - 10s 927us/sample


- #### Converting predictions to a DataFrame

In [ ]:
sub = pd.DataFrame(preds)

- #### Set column names to those generated by the one-hot encoding earlier

In [ ]:
col_names = one_hot.columns.values
sub.columns = col_names

- #### Insert the column id from the sample submission file

In [ ]:
sub.insert(0, 'id', df_test['id'])

### Predictions with pre-trained weight.

In [32]:
sub.head(5)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.003295,0.010073,0.006655,0.001397,0.005403,0.004580,0.019117,0.009447,0.015657,0.021782,0.012432,0.006409,0.001357,0.026229,0.002839,0.003175,0.010638,0.001005,0.006944,0.001849,0.001402,0.006751,0.002805,0.001610,0.015128,0.001629,0.004200,0.002014,0.003871,0.011439,0.006583,0.008266,0.007522,0.011978,0.000607,0.010303,0.007315,0.003955,0.002696,...,0.017492,0.010488,0.009110,0.005001,0.028970,0.040749,0.023176,0.015510,0.003546,0.000962,0.005938,0.001418,0.037140,0.004559,0.061934,0.002164,0.001857,0.004575,0.007565,0.012354,0.013495,0.016970,0.007115,0.002408,0.001688,0.004046,0.001751,0.002125,0.000551,0.015168,0.002914,0.001850,0.003972,0.005894,0.001424,0.008413,0.009067,0.004233,0.005832,0.013386
1,00102ee9d8eb90812350685311fe5890,0.003557,0.001760,0.004654,0.001682,0.004975,0.002723,0.007730,0.001785,0.005749,0.007642,0.010738,0.014083,0.001414,0.006594,0.005503,0.001954,0.003611,0.005245,0.004157,0.006316,0.002280,0.007380,0.002800,0.001403,0.011242,0.006891,0.002934,0.004457,0.003910,0.007762,0.009491,0.008867,0.005411,0.020947,0.003833,0.013014,0.005753,0.004338,0.001704,...,0.015992,0.004086,0.017274,0.004386,0.011053,0.027466,0.011988,0.012787,0.008959,0.005610,0.004832,0.001318,0.010672,0.005466,0.020205,0.002276,0.007874,0.005073,0.019006,0.015882,0.022870,0.014634,0.004497,0.002825,0.007858,0.004461,0.006161,0.002990,0.014276,0.007284,0.008205,0.005989,0.005559,0.006060,0.006399,0.006834,0.020459,0.008513,0.002145,0.002953
2,0012a730dfa437f5f3613fb75efcd4ce,0.005690,0.025505,0.005594,0.011083,0.002817,0.008095,0.011933,0.007694,0.006999,0.006187,0.013324,0.004056,0.007396,0.008796,0.008430,0.006140,0.007109,0.006565,0.006203,0.008343,0.007632,0.007475,0.003423,0.006148,0.005868,0.004081,0.018638,0.008806,0.005144,0.005327,0.005144,0.004055,0.013826,0.008737,0.003818,0.010600,0.007538,0.008770,0.006291,...,0.004677,0.006526,0.007149,0.007130,0.007390,0.007296,0.011114,0.003551,0.008402,0.010372,0.008928,0.004255,0.007639,0.010762,0.010714,0.003691,0.011524,0.012097,0.008209,0.006693,0.006070,0.015903,0.011199,0.007155,0.003196,0.008665,0.010056,0.012476,0.009351,0.013005,0.004552,0.008094,0.006391,0.008337,0.008658,0.014715,0.005416,0.009645,0.011179,0.010194
3,001510bc8570bbeee98c8d80c8a95ec1,0.007939,0.010598,0.003670,0.006984,0.005918,0.005126,0.006056,0.015743,0.001414,0.004458,0.009349,0.008005,0.009284,0.005038,0.018982,0.002256,0.007330,0.007330,0.010116,0.005640,0.016282,0.006722,0.003113,0.008094,0.002922,0.008009,0.009726,0.004785,0.009717,0.005435,0.002857,0.005875,0.009192,0.008229,0.011335,0.002590,0.001494,0.007554,0.011126,...,0.014079,0.020567,0.002718,0.012525,0.002474,0.001747,0.004789,0.005999,0.008278,0.013972,0.007615,0.005279,0.003601,0.008443,0.006433,0.011960,0.009164,0.007739,0.008486,0.005397,0.002835,0.007550,0.006284,0.007561,0.004664,0.013253,0.006855,0.006888

### Predictions without the Pre-Trained weights.

In [28]:
sub.head(5)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.000064,0.013971,0.000135,0.000858,0.003813,0.000423,0.000556,0.006695,0.000523,0.014394,0.020592,0.000400,0.000061,0.064436,0.000040,0.001574,0.001759,0.000199,0.035655,0.001163,0.000178,0.006584,0.000021,0.001487,0.001428,0.001147,0.003018,0.002269,0.000165,0.000792,0.005230,0.028647,0.008660,0.004092,0.000036,0.017674,0.001481,0.005466,0.000073,...,0.001257,0.000970,0.076155,0.005200,0.018806,0.008374,0.007881,0.008280,0.011740,0.000032,0.000139,0.000003,0.000948,0.015982,0.035886,0.000059,0.000302,0.000247,0.052842,0.016581,0.017982,0.006419,0.001365,0.005385,0.000108,0.001462,0.000705,0.000497,0.000016,0.002646,0.008935,0.006944,0.000072,0.005126,0.007172,0.018828,0.003084,0.002796,0.032529,0.001534
1,00102ee9d8eb90812350685311fe5890,0.000102,0.001958,0.000022,0.000154,0.000171,0.000094,0.001605,0.001420,0.009206,0.037322,0.000175,0.000126,0.000005,0.000395,0.001425,0.000085,0.000022,0.008783,0.000034,0.000810,0.000136,0.000033,0.000029,0.028797,0.002731,0.000022,0.018004,0.002664,0.000070,0.005708,0.008304,0.019577,0.002829,0.059393,0.000015,0.002430,0.000218,0.000470,0.000019,...,0.000470,0.000639,0.007300,0.004510,0.000353,0.000925,0.009064,0.017134,0.001550,0.000014,0.003908,0.000009,0.001247,0.002791,0.017626,0.000193,0.000664,0.000056,0.001602,0.018986,0.004305,0.000147,0.000597,0.000834,0.000006,0.001804,0.000393,0.000138,0.000336,0.008806,0.010847,0.001776,0.004702,0.000520,0.003388,0.003639,0.043554,0.000694,0.001210,0.061125
2,0012a730dfa437f5f3613fb75efcd4ce,0.000601,0.024437,0.002293,0.000737,0.009814,0.001633,0.000901,0.016347,0.001316,0.046961,0.072647,0.008292,0.000487,0.088488,0.000440,0.002126,0.011486,0.004057,0.013830,0.000957,0.000538,0.008847,0.000359,0.001478,0.009322,0.004909,0.008207,0.005839,0.000918,0.001753,0.001628,0.006941,0.002397,0.008535,0.000345,0.012536,0.006824,0.010402,0.000531,...,0.003709,0.002533,0.031415,0.005594,0.008123,0.003223,0.003850,0.011990,0.015065,0.000133,0.001083,0.000369,0.023355,0.003397,0.016597,0.000666,0.000586,0.003290,0.008131,0.036888,0.019641,0.016742,0.014132,0.003437,0.000468,0.001876,0.003019,0.000312,0.000083,0.005384,0.005639,0.004542,0.000655,0.014175,0.007147,0.047418,0.003654,0.005983,0.043845,0.000954
3,001510bc8570bbeee98c8d80c8a95ec1,0.004299,0.026309,0.031833,0.037659,0.001723,0.008537,0.045453,0.001893,0.004110,0.002406,0.001335,0.001524,0.002575,0.000438,0.002684,0.025355,0.000656,0.015945,0.000583,0.010560,0.005364,0.002578,0.000267,0.019001,0.000296,0.001078,0.059912,0.011950,0.018248,0.001935,0.008891,0.001112,0.008133,0.002480,0.002100,0.007544,0.000334,0.002285,0.008481,...,0.012109,0.003800,0.001982,0.003689,0.004243,0.002276,0.002316,0.002342,0.014669,0.002904,0.000919,0.001038,0.001667,0.003587,0.000736,0.024009,0.014748,0.033917,0.000558,0.002138,0.006467,0.001976,0.005961,0.008453,0.008915,0.003772,0.016621,0.002521

# Thank you.